<a href="https://colab.research.google.com/github/marbortoli/GoogleColab/blob/main/TFOD_MASKRCNN_Marina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import pathlib


In [9]:
# Clone tensorflow models repository
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [10]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1691106 sha256=aa1d878d5679c42c4fdc8439dd1c74d985522d503148c4618a1800cb80044a16
  Stored in directory: /tmp/pip-ephem-wheel-cache-2vnm26_0/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


## import dataset with images and annotations (in train and test folders)

In [ ]:
# import dataset with images and annotations (in train and test folders)

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls gdrive/MyDrive/dataset


#dataset = "./gdrive/My Drive/dataset"
#!wget -q --show-progress 'https://drive.google.com/drive/folders/1ZGzVo2zeDqmSjyY2BbfBMJebELQAU_FN?usp=sharing'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
test  train


In [ ]:
# v7 database
!pip install darwin-py

In [ ]:
# authenticate
import darwin

!darwin authenticate
# key yjoOEYG.Q9JUmWG2o7jfqQ-SV3eLlRnzrO01Y7Yr

In [ ]:
# import images from v7
!darwin dataset pull energy-machines-cloud-object-detection/scada-hmi_01:exportdata0145

In [ ]:
!python create_coco_tf_record.py --logtostderr --train_image_dir=microcontroller-segmentation/train --test_image_dir=microcontroller-segmentation/test --train_annotations_file=microcontroller-segmentation/train.json --test_annotations_file=microcontroller-segmentation/test.json --output_dir=./


In [ ]:
!python '/content/gdrive/MyDrive/coco_tf_record.py' \
  --train_image_dir='/content/gdrive/MyDrive/dataset/train/images/' \
  --test_image_dir='/content/gdrive/MyDrive/dataset/test/images/' \
  --train_annotations_file='/content/gdrive/MyDrive/dataset/train/annotations/train.json' \
  --test_annotations_file='/content/gdrive/MyDrive/dataset/test/annotations/test.json' \
  --logtostderr --output_dir='/content/./'

2022-04-26 15:48:00.390889: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
INFO:tensorflow:Found groundtruth annotations. Building annotations index.
I0426 15:48:00.863517 140702985107328 coco_tf_record.py:194] Found groundtruth annotations. Building annotations index.
INFO:tensorflow:0 images are missing annotations.
I0426 15:48:00.864549 140702985107328 coco_tf_record.py:207] 0 images are missing annotations.
INFO:tensorflow:On image 0 of 105
I0426 15:48:00.864963 140702985107328 coco_tf_record.py:212] On image 0 of 105


In [17]:
# run tf_records script for JSON annotation data

!python /content/gdrive/MyDrive/coco_tf_record.py  --logtostderr --train_image_dir = /content/gdrive/MyDrive/dataset/train/images --test_image_dir = /content/gdrive/MyDrive/dataset/test/images --train_annotations_file = /content/gdrive/MyDrive/dataset/train/annotations --test_annotations_file = /content/gdrive/MyDrive/dataset/test/annotations --output_dir=./

2022-04-26 15:47:30.868610: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/coco_tf_record.py", line 246, in <module>
    tf.app.run()
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/platform/app.py", line 36, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 312, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 258, in _run_main
    sys.exit(main(argv))
  File "/content/gdrive/MyDrive/coco_tf_record.py", line 237, in main
    FLAGS.include_masks)
  File "/content/gdrive/MyDrive/coco_tf_record.py", line 186, in _create_tf_record_from_coco_annotations
    groundtruth_data = json.load(fid)
  File "/usr/lib/python3.7/json/__init__.py", line 293, in load
    return loads(fp.read(

In [ ]:
train_record_path = '/content/train.record'
test_record_path = '/content/test.record'
labelmap_path = '/content/labelmap.pbtxt'

In [ ]:
# get ready labelmap
## The id number of each item should match the ids inside the train.json and test.json files.

!wget https://github.com/marbortoli/GoogleColab/blob/main/scada_label_map.pbtxt

--2022-04-22 08:19:27--  https://github.com/marbortoli/GoogleColab/blob/main/scada_label_map.pbtxt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘scada_label_map.pbtxt’

scada_label_map.pbt     [ <=>                ] 147.84K  --.-KB/s    in 0.01s   

2022-04-22 08:19:27 (11.7 MB/s) - ‘scada_label_map.pbtxt’ saved [151393]



## configure training

In [ ]:
# model basic parameters configuration

batch_size = 4
num_steps = 3000
num_eval_steps = 1000

In [ ]:
# select pretrained model from tensorflow model zoo
#  Tensorflow 2 Detection Model Zoo
## I want mask_rcnn to make use of the json quality

!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz
!tar -xf mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz

In [ ]:
fine_tune_checkpoint = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0'

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config

base_config_path = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config'

In [ ]:
# edit configuration file

import re

with open(base_config_path) as f:
    config = f.read()

with open('model_config.config', 'w') as f:
  
  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(4), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  # Set fine-tune checkpoint type to detection
  config = re.sub('fine_tune_checkpoint_type: "classification"', 
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  
  f.write(config)

In [ ]:
%cat model_config.config

In [ ]:
model_dir = 'training/'
pipeline_config_path = 'model_config.config'

## train custom model

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr

In [ ]:
# visualize model efficiency 

%load_ext tensorboard
%tensorboard --logdir '/content/training/train'

## export model inference graph

In [ ]:
output_directory = 'inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

In [ ]:
# download

from google.colab import files
files.download(f'/content/{output_directory}/saved_model/saved_model.pb') 